In [3]:
pip install gmaps


     ---------------------------------------- 0.0/1.1 MB ? eta -:--:--
     ----------------------------- ---------- 0.8/1.1 MB 16.6 MB/s eta 0:00:01
     ---------------------------------------- 1.1/1.1 MB 16.8 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Obtaining dependency information for geojson>=2.0.0 from https://files.pythonhosted.org/packages/8e/1b/4f57660aa148d3e3043d048b7e1ab87dfeb85204d0fdb5b4e19c08202162/geojson-3.1.0-py3-none-any.whl.metadata
  Created wheel for gmaps: filename=gmaps-0.9.0-py2.py3-none-any.whl size=2076102 sha256=e3bf2bf63ceb71ed95d82f421202095e050d577e094a6428b805ee26950f1a9b
  Stored in directory: c:\users\pranj\appdata\local\pip\cache\wheels\8e\a8\7a\361549607c6e53639877e3ef1733fe7f1b38e80df861041e3f
Successfully built gmaps
Note: you may need to restart the kernel to use updated packages.


In [4]:

pip install ipywidgets


Note: you may need to restart the kernel to use updated packages.


In [3]:
import gmaps
import gmaps.datasets
import pandas as pd
import ipywidgets as widgets
from IPython.display import display


In [4]:
mdata_02 = pd.read_csv('C:/Users/pranj/Downloads/DCU/DCU_mobiles_01-04-2022_31-07-2022.csv', sep=';')

In [5]:
mdata_01 = pd.read_csv('C:/Users/pranj/Downloads/DCU/DCU_mobiles_01-10-2021_30-01-2022.csv', sep=';')

In [6]:
mdata_df = pd.concat([mdata_01, mdata_02])

In [7]:
mdata_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8029809 entries, 0 to 2790610
Data columns (total 6 columns):
 #   Column        Dtype  
---  ------        -----  
 0   timestamp     int64  
 1   sensor ID     int64  
 2   pm2.5(ug/m3)  float64
 3   pm10(ug/m3)   float64
 4   latitude      float64
 5   longitude     float64
dtypes: float64(4), int64(2)
memory usage: 428.8 MB


In [8]:
mdata_df['date_time'] = pd.to_datetime(mdata_df['timestamp'], unit='s')

In [9]:
mdata_df.head()

,timestamp,sensor ID,pm2.5(ug/m3),pm10(ug/m3),latitude,longitude,date_time
0,1633046403,869170034050501,1.33,1.33,53.320283,-6.363592,2021-10-01 00:00:03
1,1633046405,869170034040783,2.46,3.69,53.388120,-6.171005,2021-10-01 00:00:05
2,1633046407,869170034084252,1.42,1.42,53.320965,-6.361845,2021-10-01 00:00:07
3,1633046417,869170034040783,2.69,6.15,53.388120,-6.171005,2021-10-01 00:00:17
4,1633046418,869170034084252,1.75,1.75,53.320965,-6.361845,2021-10-01 00:00:18


In [10]:
mdata_df = mdata_df.sort_values(by=['date_time'])

In [11]:
mdata_df['day_name'] = mdata_df['date_time'].dt.day_name()
mdata_df['year'] = mdata_df['date_time'].dt.year
mdata_df['month'] = mdata_df['date_time'].dt.month
mdata_df['hour'] = mdata_df['date_time'].dt.hour
mdata_df['day'] = mdata_df['date_time'].dt.day

In [12]:
mdata_df[mdata_df['hour'] == 00].max()

timestamp                1659229198
sensor ID           869170034090531
pm2.5(ug/m3)                  99.75
pm10(ug/m3)                  160.09
latitude                  54.222328
longitude                       0.0
date_time       2022-07-31 00:59:58
day_name                  Wednesday
year                           2022
month                            12
hour                              0
day                              31
dtype: object

In [13]:
mdata_df.head()

,timestamp,sensor ID,pm2.5(ug/m3),pm10(ug/m3),latitude,longitude,date_time,day_name,year,month,hour,day
0,1633046403,869170034050501,1.33,1.33,53.320283,-6.363592,2021-10-01 00:00:03,Friday,2021,10,0,1
1,1633046405,869170034040783,2.46,3.69,53.388120,-6.171005,2021-10-01 00:00:05,Friday,2021,10,0,1
2,1633046407,869170034084252,1.42,1.42,53.320965,-6.361845,2021-10-01 00:00:07,Friday,2021,10,0,1
3,1633046417,869170034040783,2.69,6.15,53.388120,-6.171005,2021-10-01 00:00:17,Friday,2021,10,0,1
4,1633046418,869170034084252,1.75,1.75,53.320965,-6.361845,2021-10-01 00:00:18,Friday,2021,10,0,1


In [16]:
import folium
from folium.plugins import HeatMap
import pandas as pd


In [ ]:
import folium
from folium.plugins import HeatMap

# Filter data for Dublin coordinates
dublin_latitude_range = (53.2, 53.5)  # Adjust latitude range as needed
dublin_longitude_range = (-6.5, -6.1)  # Adjust longitude range as needed

dublin_data_df = mdata_df[
    (mdata_df['latitude'].between(*dublin_latitude_range)) &
    (mdata_df['longitude'].between(*dublin_longitude_range))
]

#  a base map centered at the mean of the coordinates
m = folium.Map(location=[dublin_data_df['latitude'].mean(), dublin_data_df['longitude'].mean()], zoom_start=6)

# color gradient for PM2.5
pm25_gradient = {0.2: 'blue', 0.4: 'cyan', 0.6: 'lime', 0.8: 'yellow', 1.0: 'red'}

#  a HeatMap layer for pm2.5 with custom colors
heat_data_pm25 = [[row['latitude'], row['longitude'], row['pm2.5(ug/m3)']] for index, row in dublin_data_df.iterrows()]
HeatMap(heat_data_pm25, name='PM2.5', gradient=pm25_gradient).add_to(m)

#  color gradient for PM10
pm10_gradient = {0.2: 'purple', 0.4: 'black', 0.6: 'green', 0.8: 'orange', 1.0: 'brown'}

#  a HeatMap layer for pm10 with custom colors
heat_data_pm10 = [[row['latitude'], row['longitude'], row['pm10(ug/m3)']] for index, row in dublin_data_df.iterrows()]
HeatMap(heat_data_pm10, name='PM10', gradient=pm10_gradient).add_to(m)

# the map with LayerControl for switching between PM2.5 and PM10
folium.LayerControl(collapsed=False).add_to(m)

# Save the map to an HTML file or display it in Jupyter Notebook
m.save("dublin_air_quality_map2.html")
m
